### ideas
train and validate
then tale some custom images, resize and classify them

### import and parameter

In [5]:
import shutil
import os
import tqdm as notebook_tqdm
import csv
from PIL import Image
from datasets import load_dataset

### download from hugging face
put into /silver as the images are preprocessed: already properly named and resized

In [6]:
CLASSES = {
    "pizza": 76,
    "spaghetti_bolognese": 90,
    "spaghetti_carbonara": 91
}
OUT_DIR = "../data/silver"
SPLITS = ["train", "validation"]
label_names = None

In [ ]:
# Remove existing images in output folders
for split in SPLITS:
    split_dir = os.path.join(OUT_DIR, split)
    if os.path.exists(split_dir):
        for fname in os.listdir(split_dir):
            if fname.endswith(".jpg"):
                os.remove(os.path.join(split_dir, fname))

N_SPLIT = {"train": 30, "validation": 10}

for split in SPLITS:
    ds = load_dataset("ethz/food101", split=split)
    split_dir = os.path.join(OUT_DIR, split)
    os.makedirs(split_dir, exist_ok=True)

    saved_counts = {cls: 0 for cls in CLASSES}

    for idx, ex in enumerate(ds):
        label_id = ex["label"]
        if label_id in CLASSES.values():
            label = label_names[label_id]
            if saved_counts[label] < N_SPLIT[split]:
                fname = f"{idx:06d}_{label_id:02d}_{label}.jpg"
                path = os.path.join(split_dir, fname)
                ex["image"].save(path)
                saved_counts[label] += 1
                print(f"Saved {split}: {path}")

        if all(saved_counts[label_names[c]] >= N_SPLIT[split] for c in CLASSES.values()):
            break

    print(f"Done {split}! Saved:", saved_counts)

In [8]:
import os, csv
from datasets import load_dataset

CLASSES = {  # name -> id (as you had)
    "pizza": 76,
    "spaghetti_bolognese": 90,
    "spaghetti_carbonara": 91,
}
OUT_DIR = "../data/silver"
SPLITS = ["train", "validation"]
N_SPLIT = {"train": 50, "validation": 20}  # example targets per class

# --- label metadata (once) ---
label_names = load_dataset("ethz/food101", split="train").features["label"].names
id_to_name = {i: n for i, n in enumerate(label_names)}
target_ids = set(CLASSES.values())
target_names = {id_to_name[i] for i in target_ids}  # {"pizza", "spaghetti_bolognese", ...}

for split in SPLITS:
    split_dir = os.path.join(OUT_DIR, split)
    os.makedirs(split_dir, exist_ok=True)
    csv_path = os.path.join(split_dir, "labels.csv")

    ds = load_dataset("ethz/food101", split=split)
    saved_counts = {name: 0 for name in target_names}
    rows = []

    for idx, ex in enumerate(ds):
        lid = ex["label"]
        if lid in target_ids:
            label = id_to_name[lid]  # class name
            if saved_counts[label] < N_SPLIT[split]:
                fname = f"{idx:06d}_{lid:02d}_{label}.jpg"
                path = os.path.join(split_dir, fname)
                ex["image"].save(path)
                rows.append([fname, label])
                saved_counts[label] += 1

        # stop when ALL target classes reached their per-split quota
        if all(count >= N_SPLIT[split] for count in saved_counts.values()):
            break

    with open(csv_path, "w", newline="") as f:
        w = csv.writer(f)
        w.writerow(["filename", "label"])
        w.writerows(rows)

    print(f"Done {split}! Saved: {saved_counts}  Labels CSV: {csv_path}")


/usr/local/python/3.11.13/lib/python3.11/site-packages/PIL/TiffImagePlugin.py:950: UserWarning: Truncated File Read
  warnings.warn(str(msg))


Done train! Saved: {'spaghetti_bolognese': 50, 'spaghetti_carbonara': 50, 'pizza': 50}  Labels CSV: ../data/silver/train/labels.csv
Done validation! Saved: {'spaghetti_bolognese': 20, 'spaghetti_carbonara': 20, 'pizza': 20}  Labels CSV: ../data/silver/validation/labels.csv
